In [1]:
import pandas as pd
import numpy as np
import os

DATA_PATH = "/workspace/lg/2022-lguplus-AI-Ground/data"

In [64]:
df_history = pd.read_csv(os.path.join(DATA_PATH, 'history_data.csv')).drop_duplicates(['profile_id', 'ss_id', 'log_time', 'album_id'], keep='last')

In [65]:
df_history['continuous_play'] = np.where(df_history['continuous_play']=='Y', 1, 0)
df_history['short_trailer'] = np.where(df_history['short_trailer']=='Y', 1, 0)

df_history['ss_continuous_cnt'] = df_history.groupby(['profile_id', 'ss_id'])['continuous_play'].transform('cumsum')


In [66]:
df_watch = pd.read_csv(os.path.join(DATA_PATH, 'watch_e_data.csv'))

df_watch['watch_ratio'] = df_watch['watch_time'] / df_watch['total_time']

In [67]:
# 나중에 평점 높게 처리할 것

# df_watch.duplicated(['profile_id', 'ss_id', 'album_id'])



In [68]:
# threshold = 0.1

# df_watch_real = df_watch.loc[(df_watch['watch_ratio'] >= 0.7 )]

In [69]:
df_seen = pd.merge(df_history.drop('log_time', axis=1),
                   df_watch.drop('log_time', axis=1),
                   how='inner',
                   on=['profile_id', 'ss_id', 'album_id'])

In [70]:
df_seen['watch_ratio'] = df_seen['watch_ratio'].apply(lambda x: 1 if x >= 0.3 else 0)


In [71]:
#짧은 걸 여러 번 보는 것 -> preference positive하게

df_buy = pd.read_csv(os.path.join(DATA_PATH, 'watch_e_data.csv'), index_col=False).drop_duplicates(['profile_id', 'ss_id', 'log_time', 'album_id'], keep='last')

df_buy['watch_ratio'] = df_buy['watch_time'] / df_buy['total_time']

df_buy_p = df_buy[df_buy['watch_ratio'] > 0.1]

df_buy_p['watch_ratio'] = df_buy['watch_ratio'].apply(lambda x: 1)

df_buy_p = df_buy_p[['profile_id', 'album_id', 'watch_ratio', 'ss_id']]

<ipython-input-71-44b6634dc7b4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_buy_p['watch_ratio'] = df_buy['watch_ratio'].apply(lambda x: 1)


In [72]:
df_search = pd.read_csv(os.path.join(DATA_PATH, 'search_data.csv'), index_col=False).drop_duplicates(['profile_id', 'ss_id', 'log_time', 'album_id'], keep='last')

df_search['watch_ratio'] = 1

In [73]:
df = pd.merge(df_seen[['profile_id', 'album_id', 'watch_ratio', 'ss_id']], 
              df_buy_p[['profile_id', 'album_id', 'watch_ratio', 'ss_id']],
              how='outer')

In [74]:
df = pd.merge(df, 
              df_search[['profile_id', 'album_id', 'watch_ratio', 'ss_id']],
              how='outer')

,profile_id,album_id,watch_ratio,ss_id


In [70]:
df.columns = ["userID", "itemID", "rating", "timestamp"]


# .inter file 생성

TARGET_DIR = os.path.join(os.getcwd(), "../data/recbole")
TARGET_NAME = "recbole.inter"

os.makedirs(TARGET_DIR, exist_ok=True)

df = df.rename(
    columns={
        "userID": "user_id:token",
        "itemID": "item_id:token",
        "rating": "rating:float",
        "timestamp": "timestamp:float",
    }
)

df.to_csv(os.path.join(TARGET_DIR, TARGET_NAME), index=False, sep="\t")

print("Done!")



Done!
